In [ ]:
"""
    모델의원리이해 :
        모델의주요layer의의미, 구성방법, 동작원리, 활용방법에대해이해를하고있나
        모델의 결과와 정답을 이용하여 학습으로 연결시킬 수 있나
"""   

In [ ]:
class ViT(nn.Module):
    """Vision Transformer (ViT)
    - 이미지를 패치로 쪼개 토큰 시퀀스를 만들고
    - CLS 토큰 + 위치 임베딩을 더한 뒤
    - Transformer Encoder로 전역(Self-Attention) 관계를 학습하여
    - CLS(또는 mean pool) 표현으로 분류합니다.
    """
    def __init__(self, cfg: ViTConfig):
        super().__init__()

        # 1) 입력 해상도/패치 크기를 (H,W) 튜플로 정규화
        image_height, image_width = pair(cfg.image_size)
        patch_height, patch_width = pair(cfg.patch_size)

        # 2) 패치가 이미지에 딱 나누어 떨어져야 (h, w) 그리드가 정확히 형성됨
        assert image_height % patch_height == 0 and image_width % patch_width == 0, "Image dimensions must be divisible by the patch size."

        # 3) 패치 개수(=토큰 개수)와 한 패치의 펼친 차원 계산
        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = cfg.channels * patch_height * patch_width

        # 4) 풀링 방식 검증: CLS 토큰을 쓸지 mean pool을 쓸지 선택
        assert cfg.pool in {"cls", "mean"}, "pool must be 'cls' or 'mean'"

        # 5) 패치 토큰화: (B,C,H,W) -> (B, N, patch_dim) -> (B, N, dim)
        self.to_patch_embedding = nn.Sequential(
            # 패치 그리드로 자른 뒤, 각 패치를 1D 벡터로 펼쳐 토큰 시퀀스를 만듦
            Rearrange(
                "b c (h p1) (w p2) -> b (h w) (p1 p2 c)",
                p1=patch_height,
                p2=patch_width,
            ),
            # 펼친 패치 벡터를 Transformer hidden size(dim)로 선형 투영
            nn.Linear(patch_dim, cfg.dim),
        )

        # 6) CLS 토큰 + 위치 임베딩(학습 파라미터)
        self.cls_token = nn.Parameter(torch.randn(1, 1, cfg.dim))
        # LLM GPT의 token embedding 대신 cls_token으로 분류하는 방식인듯?
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, cfg.dim))
        self.dropout = nn.Dropout(cfg.emb_dropout)

        # 7) Transformer Encoder 스택(Attention + FFN + Residual/PreNorm)
        self.transformer = Transformer(
            dim=cfg.dim,
            depth=cfg.depth,
            heads=cfg.heads,
            dim_head=cfg.dim_head,
            mlp_dim=cfg.mlp_dim,
            dropout=cfg.dropout,
        )

        self.pool = cfg.pool

        # 8) 분류 헤드: (CLS/mean) 표현 -> LayerNorm -> Linear(logits)
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(cfg.dim),
            nn.Linear(cfg.dim, cfg.num_classes),
        )

    def forward(self, img):
        # A) 패치 임베딩으로 토큰 시퀀스 생성: (B,C,H,W) -> (B,N,dim)
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        # B) CLS 토큰을 배치만큼 복제해 시퀀스 맨 앞에 붙임: (B,1,dim)
        cls_tokens = repeat(self.cls_token, "1 1 d -> b 1 d", b=b)
        x = torch.cat((cls_tokens, x), dim=1)  # (B, N+1, dim)

        # C) 위치 임베딩을 더해 토큰 순서(공간 위치) 정보를 주입
        x = x + self.pos_embedding[:, : (n + 1)]   # (B, N+1, dim) + (1, N+1, dim) -> Broadcasting
        x = self.dropout(x)

        # D) Encoder를 통과하며 전역 의존성(Self-Attention) 학습
        x = self.transformer(x)  # (B, N+1, dim)

        # E) 이미지 표현 벡터 선택: CLS 토큰(0번) 또는 mean pooling
        x = x[:, 0] if self.pool == "cls" else x.mean(dim=1)   # (B, dim)

        # F) 최종 logits 출력 (softmax는 CrossEntropyLoss 내부에서 처리)
        return self.mlp_head(x)


In [ ]:
# =========================
# 3) U-Net 모델 정의
# =========================
# 이 섹션의 목표:
# - U-Net의 "인코더(Down) → 보틀넥 → 디코더(Up) + Skip Connection" 구조를
#   PyTorch 코드로 직접 따라가며 이해합니다.
#
# 핵심 아이디어(한 줄):
# - Down에서 공간 해상도(H,W)는 줄이고 채널(C)은 늘리면서 특징을 추출하고,
#   Up에서 해상도를 복원하면서 Down 단계의 특징맵을 Skip으로 concat하여
#   localization(위치 정보)을 되살립니다.

class DoubleConv(nn.Module):
    """U-Net 기본 블록: (Conv → ReLU) × 2

    - 첫 번째 Conv가 채널을 '중간 채널(mid_channels)'로 바꾸고,
      두 번째 Conv가 '출력 채널(out_channels)'로 맞춥니다.
    - 기본값(mid_channels=None)일 때는 mid_channels=out_channels로 두어,
      (C_in → C_out → C_out) 형태가 됩니다.

    입력/출력 텐서 형태:
      - 입력:  (B, C_in, H, W)
      - 출력:  (B, C_out, H, W)  # padding=1 이라 H,W 유지
    """
    def __init__(self, in_channels: int, out_channels: int, mid_channels: int | None = None):
        super().__init__()

        # U-Net에서 업샘플 후 concat을 하면 채널이 2배가 되므로,
        # bilinear 업샘플링을 쓸 때는 mid_channels=in_channels//2 처럼
        # '중간 채널'을 줄여주는 방식이 흔히 사용됩니다.
        if mid_channels is None:
            mid_channels = out_channels

        self.net = nn.Sequential(
            # 1) (C_in → C_mid)
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True),

            # 2) (C_mid → C_out)
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.ReLU(inplace=True),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)


In [ ]:
class Down(nn.Module):
    """Downsampling 블록: MaxPool(2)로 해상도 1/2 → DoubleConv

    입력/출력 텐서 형태:
      - 입력:  (B, C_in,  H,  W)
      - 출력:  (B, C_out, H/2, W/2)
    """
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # 1) 해상도 축소
        x = self.pool(x)           # (B, C_in, H/2, W/2)
        # 2) 채널 확장 + 특징 추출
        x = self.conv(x)           # (B, C_out, H/2, W/2)
        return x



In [ ]:
class Up(nn.Module):
    """Upsampling 블록: 업샘플링 → (Skip concat) → DoubleConv

    구현 관점(중요):
    - in_channels 는 concat 이후 채널 수를 의미합니다.
      예) x1(디코더) 채널=512, x2(skip) 채널=512 → concat 채널=1024 → in_channels=1024

    - bilinear=True:
        1) 업샘플은 파라미터 없는 bilinear interpolation으로 수행
        2) concat 후 DoubleConv에서 mid_channels를 in_channels//2 로 두어
           채널을 자연스럽게 '절반'으로 줄이는 방식(원 논문/레퍼런스 구현과 동일 계열)

    - bilinear=False:
        ConvTranspose2d로 업샘플 자체를 학습(파라미터 증가)
    """
    def __init__(self, in_channels: int, out_channels: int, bilinear: bool = False):
        super().__init__()

        if bilinear:
            # (B, C, H/2, W/2) → (B, C, H, W)
            self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)

            # bilinear일 때는 업샘플 후 채널 수(C)가 그대로 유지됩니다.
            # concat 결과(in_channels)를 DoubleConv로 처리하되,
            # 첫 conv의 출력(mid_channels)을 in_channels//2로 두어 채널을 줄입니다.
            self.conv = DoubleConv(in_channels, out_channels, mid_channels=in_channels // 2)
        else:
            # (B, C, H/2, W/2) → (B, C/2, H, W)  (deconv가 채널도 절반으로 줄여줌)
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1: torch.Tensor, x2: torch.Tensor) -> torch.Tensor:
        # x1: (B, C_dec, H/2, W/2)  / x2: (B, C_skip, H, W)

        # 1) 업샘플링: 해상도를 skip과 맞추기
        x1 = self.up(x1)  # bilinear: 채널 유지 / deconv: 채널이 절반으로 감소

        # 2) (필요 시) 패딩으로 크기 정렬
        #    - 홀수 크기 입력 등으로 인해 x1과 x2의 H/W가 1~2 픽셀 정도 다를 수 있습니다.
        diff_y = x2.size(2) - x1.size(2)
        diff_x = x2.size(3) - x1.size(3)
        x1 = F.pad(x1, [diff_x // 2, diff_x - diff_x // 2,  # left, right
                        diff_y // 2, diff_y - diff_y // 2]) # top, bottom

        # 3) 채널 방향 concat (skip 연결)
        #    cat dim=1 은 채널(C) 축
        x = torch.cat([x2, x1], dim=1)  # (B, C_skip + C_up, H, W) == (B, in_channels, H, W)

        # 4) conv 블록으로 특징 정제 + 채널 축소
        return self.conv(x)

In [ ]:
class OutConv(nn.Module):
    """마지막 1x1 conv: 채널을 클래스 수로 매핑

    예)
      - binary segmentation: n_classes=1 (logits 1채널)
      - multi-class:         n_classes=K (logits K채널)
    """
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.conv(x)

In [ ]:
class UNet(nn.Module):
    """U-Net 전체 모델

    전형적인 채널 구성 예:
      1 → 64 → 128 → 256 → 512 → 1024 (down)
      1024 → 512 → 256 → 128 → 64 (up)
      숫자 외워야 할 듯.
    """
    def __init__(self, n_channels: int, n_classes: int, bilinear: bool = False):
        super().__init__()

        # -------------------------
        # Encoder (Contracting path)
        # -------------------------
        self.inc = DoubleConv(n_channels, 64)   # (B, n_channels, H, W) → (B, 64, H, W)
        self.down1 = Down(64, 128)              # → (B, 128, H/2, W/2)
        self.down2 = Down(128, 256)             # → (B, 256, H/4, W/4)
        self.down3 = Down(256, 512)             # → (B, 512, H/8, W/8)

        # bilinear 업샘플링이면 파라미터/연산을 줄이기 위해 bottleneck 채널을 1024 대신 512로 줄이는 경우가 많음
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)  # → (B, 1024/f, H/16, W/16)

        # -------------------------
        # Decoder (Expanding path)
        # -------------------------
        # Up 블록의 in_channels는 "concat 후 채널" 기준으로 설계되어야 합니다.
        self.up1 = Up(1024, 512 // factor, bilinear)  # (skip=512)와 concat을 고려
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)

        # -------------------------
        # Output head
        # -------------------------
        self.outc = OutConv(64, n_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # -------------------------
        # Encoder: skip을 위해 각 단계 출력 저장
        # -------------------------
        x1 = self.inc(x)       # (B, 64, H, W)
        x2 = self.down1(x1)    # (B, 128, H/2, W/2)
        x3 = self.down2(x2)    # (B, 256, H/4, W/4)
        x4 = self.down3(x3)    # (B, 512, H/8, W/8)
        x5 = self.down4(x4)    # (B, 1024/f, H/16, W/16)

        # -------------------------
        # Decoder: 업샘플 + skip concat
        # -------------------------
        x = self.up1(x5, x4)   # (B, 512/f, H/8, W/8)
        x = self.up2(x, x3)    # (B, 256/f, H/4, W/4)
        x = self.up3(x, x2)    # (B, 128/f, H/2, W/2)
        x = self.up4(x, x1)    # (B, 64, H, W)

        # -------------------------
        # 최종 logits 출력
        # -------------------------
        logits = self.outc(x)  # (B, n_classes, H, W)
        return logits
